In [12]:
import hnn_core
from hnn_core import calcium_model, read_params, simulate_dipole, average_dipoles
from hnn_core.viz import plot_dipole
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import pickle
import numpy as np

%matplotlib inline

In [13]:
#fparams = "param_files/default.param"
fparams = "param_files/TEP_from_def_Liz_2newInputs_optimized_TMS100.param"
params = read_params(fparams)
print(params)
#print(len(params))

{
    "L2Basket_Gauss_A_weight": 0.0,
    "L2Basket_Gauss_mu": 2000.0,
    "L2Basket_Gauss_sigma": 3.6,
    "L2Basket_Pois_A_weight_ampa": 0.0,
    "L2Basket_Pois_A_weight_nmda": 0.0,
    "L2Basket_Pois_lamtha": 0.0,
    "L2Pyr_Gauss_A_weight": 0.0,
    "L2Pyr_Gauss_mu": 2000.0,
    "L2Pyr_Gauss_sigma": 3.6,
    "L2Pyr_Pois_A_weight_ampa": 0.0,
    "L2Pyr_Pois_A_weight_nmda": 0.0,
    "L2Pyr_Pois_lamtha": 0.0,
    "L2Pyr_ampa_e": 0.0,
    "L2Pyr_ampa_tau1": 0.5,
    "L2Pyr_ampa_tau2": 5.0,
    "L2Pyr_apical1_L": 306.0,
    "L2Pyr_apical1_diam": 4.08,
    "L2Pyr_apicaloblique_L": 340.0,
    "L2Pyr_apicaloblique_diam": 3.91,
    "L2Pyr_apicaltrunk_L": 59.5,
    "L2Pyr_apicaltrunk_diam": 4.25,
    "L2Pyr_apicaltuft_L": 238.0,
    "L2Pyr_apicaltuft_diam": 3.4,
    "L2Pyr_basal1_L": 85.0,
    "L2Pyr_basal1_diam": 4.25,
    "L2Pyr_basal2_L": 255.0,
    "L2Pyr_basal2_diam": 2.72,
    "L2Pyr_basal3_L": 255.0,
    "L2Pyr_basal3_diam": 2.72,
    "L2Pyr_dend_Ra": 200.0,
    "L2Pyr_dend_cm": 0.619

In [14]:
tep_evoked = ['dist_1', 'dist_2', 'dist_3', 'prox_1', 'prox_2', 'prox_3']
erp_evoked = ['dist_4', 'prox_4', 'prox_5']

conductance = 0
spikes = 0
synchrony = 1

# Conductance scaling factors for a range of %MT
TMS_intensity_base = 80 # %MT for original data
mts = [80, 100, 120, 160, 200]
TMS_intensity_scalars = [mt/TMS_intensity_base for mt in mts]

num_spikes = [1, 2, 3]

sigma_scalars = [1, 0.8, 0.6, 0.4]

vary = {
    'conductance': [conductance, mts],
    'spikes': [spikes, num_spikes],
    'synchrony': [synchrony, sigma_scalars],
}

if conductance:
    params_list = [deepcopy(params) for _ in TMS_intensity_scalars]
    for par, scalar in zip(params_list, TMS_intensity_scalars):
        for p in par.keys():
            if any([f'gbar_ev{tep}' in p for tep in tep_evoked]):
                par[p] *= scalar

if spikes:
    params_list = [deepcopy(params) for _ in num_spikes]
    for par, spike in zip(params_list, num_spikes):
        par['numspikes_evdist_2'] = spike # Only the first evoked input gets more spikes, which is evdist_2 for some reason

if synchrony:
    params_list = [deepcopy(params) for _ in sigma_scalars]
    for par, scalar in zip(params_list, sigma_scalars):
        for p in par.keys():
            if any([f'sigma_t_ev{tep}' in p for tep in tep_evoked]):
                par[p] *= scalar

print([f'sigma_t_ev{tep}' for tep in tep_evoked])
print(params_list)


['sigma_t_evdist_1', 'sigma_t_evdist_2', 'sigma_t_evdist_3', 'sigma_t_evprox_1', 'sigma_t_evprox_2', 'sigma_t_evprox_3']
[{
    "L2Basket_Gauss_A_weight": 0.0,
    "L2Basket_Gauss_mu": 2000.0,
    "L2Basket_Gauss_sigma": 3.6,
    "L2Basket_Pois_A_weight_ampa": 0.0,
    "L2Basket_Pois_A_weight_nmda": 0.0,
    "L2Basket_Pois_lamtha": 0.0,
    "L2Pyr_Gauss_A_weight": 0.0,
    "L2Pyr_Gauss_mu": 2000.0,
    "L2Pyr_Gauss_sigma": 3.6,
    "L2Pyr_Pois_A_weight_ampa": 0.0,
    "L2Pyr_Pois_A_weight_nmda": 0.0,
    "L2Pyr_Pois_lamtha": 0.0,
    "L2Pyr_ampa_e": 0.0,
    "L2Pyr_ampa_tau1": 0.5,
    "L2Pyr_ampa_tau2": 5.0,
    "L2Pyr_apical1_L": 306.0,
    "L2Pyr_apical1_diam": 4.08,
    "L2Pyr_apicaloblique_L": 340.0,
    "L2Pyr_apicaloblique_diam": 3.91,
    "L2Pyr_apicaltrunk_L": 59.5,
    "L2Pyr_apicaltrunk_diam": 4.25,
    "L2Pyr_apicaltuft_L": 238.0,
    "L2Pyr_apicaltuft_diam": 3.4,
    "L2Pyr_basal1_L": 85.0,
    "L2Pyr_basal1_diam": 4.25,
    "L2Pyr_basal2_L": 255.0,
    "L2Pyr_basal2_diam"

In [15]:
nets = [calcium_model(params=par, add_drives_from_params=True) for par in params_list]

In [16]:
dpls_list = [simulate_dipole(net, tstop=params["tstop"], n_trials=3) for net in nets]


Joblib will run 3 trial(s) in parallel by distributing trials over 1 jobs.
Building the NEURON model
[Done]
Trial 1: 0.03 ms...
Trial 1: 10.0 ms...
Trial 1: 20.0 ms...
Trial 1: 30.0 ms...
Trial 1: 40.0 ms...
Trial 1: 50.0 ms...
Trial 1: 60.0 ms...
Trial 1: 70.0 ms...
Trial 1: 80.0 ms...
Trial 1: 90.0 ms...
Trial 1: 100.0 ms...
Trial 1: 110.0 ms...
Trial 1: 120.0 ms...
Trial 1: 130.0 ms...
Trial 1: 140.0 ms...
Trial 1: 150.0 ms...
Trial 1: 160.0 ms...
Trial 1: 170.0 ms...
Trial 1: 180.0 ms...
Trial 1: 190.0 ms...
Trial 1: 200.0 ms...
Trial 1: 210.0 ms...
Trial 1: 220.0 ms...
Trial 1: 230.0 ms...
Trial 1: 240.0 ms...
Trial 1: 250.0 ms...
Trial 1: 260.0 ms...
Trial 1: 270.0 ms...
Trial 1: 280.0 ms...
Trial 1: 290.0 ms...
Building the NEURON model
[Done]
Trial 2: 0.03 ms...
Trial 2: 10.0 ms...
Trial 2: 20.0 ms...
Trial 2: 30.0 ms...
Trial 2: 40.0 ms...
Trial 2: 50.0 ms...
Trial 2: 60.0 ms...
Trial 2: 70.0 ms...
Trial 2: 80.0 ms...
Trial 2: 90.0 ms...
Trial 2: 100.0 ms...
Trial 2: 110.0 ms.

In [17]:
if conductance:
    fname = 'nets_dpls_list_vary_TEP_ERP_80_100_120_160_200_mt'
if spikes:
    fname = 'nets_dpls_list_vary_TEP_ERP_1_2_3_spikes_evdist_2'
if synchrony:
    fname = 'nets_dpls_list_vary_TEP_ERP_1_08_06_04_sigma'

with open(f'sim_data/{fname}.pkl', 'wb') as file:
    pickle.dump([nets, dpls_list, vary], file)


In [18]:
# with open('dpls_list_TEP_ERP_80_100_120.pkl', 'rb') as file:
#     dpls_list_80_100_120 = pickle.load(file)

In [19]:
# print(dpls_list)
# print(dpls_list_80_100_120)
# dpls_list_all = []
# dpls_list_all.extend(dpls_list_80_100_120)
# dpls_list_all.extend(dpls_list)
# print(dpls_list_all)

In [20]:
# with open('dpls_list_TEP_ERP_80_100_120_160_200.pkl', 'wb') as file:
#     pickle.dump(dpls_list_all, file)